In [1]:
import pandas as pd

# Chromosome  1 AF

The new pipleline for getting the gnomAD AF did not finished properly on the following chromosoms:
 <a class="anchor" id="missing_chr"></a>
- chr1
- chr3
- chr5
- chr8
- chr10
- chr15

In order to fix it, I made another script, This time I used the local gnomAD vcf files (which Ofer downloaded) & gave more memory to the sbatch command (<a href=code/get_gnomAD_AF_leftovers.sh>script code can be found here</a>) : 
> ```bash
sbatch -o slurm_log/second_left_over_%A_%a --mem 300G --killable --requeue -c 16 --array=0-4 -t 56:00:00 --wrap "code/get_gnomAD_AF_leftovers.sh"

For all chromosomes, except chr1, the proccesed finished properly, while The slurm log file on chromosome 1 showed  errors: <br>

> [E::bgzf_read_block] Invalid BGZF header at offset 189854641494 <br>
[E::bgzf_uncompress] Inflate operation failed: 3<br>

The 'bgzf_read_block' error can be found many times and the second only once. <br>
these errors did not stopped the script, and there is an output file, which the maximal position in it is
<a class="anchor", id="max_pos"></a>  243,019,330 out  of 249MB.<br>
This might suggest the errors, but there are might be some regions in the middle which the pipline failed to achive. <br>

Since the same happned in the first run of the new pipeline, on the chromosome mentioned [above](#missing_chr), and now it worked properly, there are two possibilities:
1. chromosome 1 file is damaged both in AWS & in our file system
2. the --killable flag cause / not enough memory is given to the program

since the 2nd option is not only more likely, but it is the one that I can try to fix. <br>

This time I'll try to run the command w/o the killable flag.

## step 1: get chromosme 1 intervals
I'll extract chromosome 1 interval from the interval file



In [12]:
interval_file = pd.read_csv('data/peak_bcftools_format.tsv', sep='\t', header=None)
chr1_interval = interval_file[interval_file[0] == 'chr1']
chr1_interval.to_csv('data/chr1_interval.tsv', sep='\t', index=False, header=None)
print("Five ATAC interval with the maximal position in chromsome 1:")
chr1_interval.tail()

Five ATAC interval with the maximal position in chromsome 1:


,0,1,2
19176,chr1,248889240,248889941
19177,chr1,248905657,248907559
19178,chr1,248918189,248918244
19179,chr1,248923380,248925434
19180,chr1,248927609,248928681


No variants in the regions above can be found in the output of the [pipline result](#max_pos). <br>

To check if there are variants in this regions, a command was executed on 2 files:
1. the vcf on AWS server
2. the vcf on the local system

the command: 

> ```bash
bcftools query -i 'FILTER="PASS"' -r chr1:248927609-248928681 -f "%CHROM:%POS %REF-%ALT %AF %FILTER\n" FILE



the output on the vcf from the AWS resulted a list of variants (this is a sample):

chr1:248928644 G-A 1.97086e-05     PASS <br>
chr1:248928648 G-T 6.57047e-06     PASS <br>

the out put on the local vcf file :
[E::bgzf_read_block] Invalid BGZF header at offset 195393501396 <br>

This might suggest the local VCF is corrupted, Hence I'll run the same sbatch command on chromosome 1, This time with the AWS file:

> https://gnomad-public-us-east-1.s3.amazonaws.com/release/3.1.2/vcf/genomes/gnomad.genomes.v3.1.2.sites.chr1.vcf.bgz

the full script can be found <a href='code/get_gnomAD_AF_CHR1.sh'> here, the following is a summary of the command </a>

>```bash
bcftools query -i 'FILTER="PASS"' -R data/chr1_interval.tsv -f "%CHROM\t%POS\t%REF\t%ALT\t%AF\t\n" https://gnomad-public-us-east-1.s3.amazonaws.com/release/3.1.2/vcf/genomes/gnomad.genomes.v3.1.2.sites.chr1.vcf.bgz

main changes:
Runs only on chromosome 1, using interval file which contains only the regions in chromosome 1
<br>

> ```bash
sbatch -o slurm_log/chr1_gnomAD_AF_%A.log --mem 300G -c 16 -t 56:00:00 --wrap "code/get_gnomAD_AF_CHR1.sh"
Submitted batch job 14402090


![image](http://www.quickmeme.com/img/80/8006789bb0cc3734ca56a33a79d2660023d66fd71ea1755948161b32292801bf.jpg)

Update- The process finished succesfully :) (6/03/23) <br>
Happy Purim 